<h1 style="color:gold">Bonus: Simon's Algorithm</h1>

In [82]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.visualization import plot_distribution

import math as m
import matplotlib

quasm_sim = AerSimulator()

Simons algorithm is about finding a "hidden" subset in a group.

We are seaching for `s` such that:
- f(`x`) = f(`x` XOR `s`)
- `s` and `x` belonging to `{0, 1}**n`
- `s` != 0

A simplified description of the algorithm goes like this:
- Superpose the system
- Send the system to an Oracle f(x) that uses the given register to compute f(x), we ignore the result
- Measure the state of the `x` register once it went through the Oracle function, it will be in a superposition of `x` and `x^s`, measuring will yield either one
- Collect `k` strings throuh `k` iterations to build a set of len (n-1) of linearly independent strings
- You now have a system of linear equations that once solved will give `s`

The reason is:
- Since to be valid the oracle is a 2-to-1 function, there will be constructive interference where `x` and `x^s` will become the state with the highest amplitude for any `x` over the range

In [83]:
def simon_oracle(qc, n, s):
    s = s[::-1] # flip for qiskit
    for i in range(n):
        if s[i] == '1':
            qc.cx(i, n + i )

In [84]:
def execute_quantum_subroutine(qc, shots=500, method='aer'):
    if method == 'aer':
        qc_aer = qiskit.transpile(qc, backend=quasm_sim)
        counts = quasm_sim.run(qc_aer, shots=shots).result().get_counts()
        return counts
    else:
        raise ValueError("Method not supported")


In [85]:
def build_circuit(s):
    """
    |x> register, register of search space
    |z> register, register of oracle output
    """
    n = len(s)

    xq_register = QuantumRegister(n, name='x')
    zq_register = QuantumRegister(n, name='z')
    xc_register = ClassicalRegister(n, name='c')

    qc = QuantumCircuit(xq_register, zq_register, xc_register)

    # Superpose |x> register
    qc.h(xq_register)

    # Use the |x> register as input to the oracle
    # It's state will become a superposition of `x` and `x^s`
    qc.barrier()
    simon_oracle(qc, n, s)
    qc.barrier()

    # Separate |x> register
    qc.h(xq_register)
    qc.measure(xq_register, xc_register)

    return qc


In [86]:
s = '00101' # The hidden substring to find

n = len(s)
simon_circuit = build_circuit(s)
print(f"Depth: {simon_circuit.depth()}")
# simon_circuit.draw()

Depth: 4


In [87]:
shots = 200 * n

counts = execute_quantum_subroutine(simon_circuit, shots)

print(counts)

{'000': 291, '001': 309}


Final step:
- Create a set with `n-1` linearly independent  bitstrings.
- This allows for a system of linear equations to find `s`
- If the system has more than 1 or no solution the oracle is invalid

#pain #sufferwithme

In [88]:
def str_overlap(a, b):
    for i in range(len(a)):
        if b[i] == a[i] and a[i] == '1':
            return True
    return False


def build_set(counts, n, shots=200*n):
    """
    Builds a linearly independent set of bitstrings from the quantum subroutines.
    """
    noise_threshold = max(1, shots // max(1, len(counts)) // 15)  # Avoid division by zero
    li_set = set()

    for key in counts.keys():
        if key == ('0' * n):
            continue
        if counts[key] > noise_threshold:  # Filter out noisy results
            # Check if the key is linearly independent from the current set
            independent = True
            for set_string in li_set:
                if str_overlap(key, set_string):  # Linearly dependent if XOR gives all zeros
                    independent = False
                    break
            # Add the key if it's independent
            if independent:
                li_set.add(key)
            # Stop if we have n-1 independent bitstrings
            if len(li_set) == n - 1:
                return li_set

    return li_set  # In case fewer than n-1 independent strings are found

In [89]:
li_set = build_set(counts, n, shots)
print(li_set)

s_found = '0' * n
for string in li_set:
    for i in range(n):
        if string[i] == '1':
            # s_found[i] = '1' # Not allowed
            s_found = s_found[:i] + '1' + s_found[i+1:]

print(f"The hidden string was computed as: {s_found}")
print(f"The real answer is: {s} it is thus {'correct' if s_found == s else 'incorect'}")


{'001'}
The hidden string was computed as: 001
The real answer is: 001 it is thus correct
